**Сегментация трилогии, препроцессинг и NER**

In [1]:
import re
import pandas as pd
from pathlib import Path
import pickle

# Для предобработки и токенизации
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

# Для лемматизации
import pymorphy3
from pymorphy3 import MorphAnalyzer

# Для извлечения сущностей
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsNERTagger,
    NewsMorphTagger,
    Doc
)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Huawei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def split_by_chapters(text):
    """Ищем в тексте вхождения слова 'Глава' и делим текст по ним"""
    chapter_pattern = r'(\n\s*Глава\s+(?:[IVX]+|\d+))'

    # Разрезаем текст, сохраняя сами заголовки внутри списка
    parts = [p for p in re.split(chapter_pattern, text) if p]
    
    segments = []
    current_segment = ""
    
    for part in parts:
        # Если часть текста совпадает с паттерном главы, начинаем новый сегмент
        if re.match(chapter_pattern, part):
            # Игнорируем слишком короткие фрагменты (менее 50 слов)
            if current_segment.strip():
                if len(current_segment.split()) > 50: 
                    segments.append(current_segment)
            current_segment = part
        else:
            # Если это обычный текст, приклеиваем его к текущей главе
            current_segment += " " + part
    # Добавляем последнюю главу        
    if current_segment.strip() and len(current_segment.split()) > 50:
        segments.append(current_segment)
        
    return segments

def load_and_split_text(file_path):
    """Делим весь файл на 3 основные книги и затем на главы"""
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Поиск границ между томами (Хранители, Две Твердыни, Возвращение Государя)
    book_markers = re.finditer(r'\bХРАНИТЕЛИ\b|\n\bДВЕ\s+ТВЕРДЫНИ\b|\n\bВОЗВРАЩЕНИЕ\s+ГОСУДАРЯ\b', text)
    positions = [m.start() for m in book_markers]
    print(f"Найдены границы книг на позициях: {positions}")
    
    book1_text = text[positions[0]:positions[1]]
    book2_text = text[positions[1]:positions[2]]
    book3_text = text[positions[2]:len(text)]
    
    # Нарезка каждой книги на главы
    segments_book1 = split_by_chapters(book1_text)
    segments_book2 = split_by_chapters(book2_text)
    segments_book3 = split_by_chapters(book3_text)
    
    print(f"\nХранители: {len(segments_book1)} сегментов")
    print(f"Две твердыни: {len(segments_book2)} сегментов")
    print(f"Возвращение государя: {len(segments_book3)} сегментов")
    
    segments = segments_book1 + segments_book2 + segments_book3
    print(f"Всего смысловых сегментов (глав): {len(segments)}")

    # Вычисляем, где заканчивается одна книга и начинается другая
    book1_end = len(segments_book1)
    book2_end = book1_end + len(segments_book2)
    
    print(f"\nГраницы:")
    print(f"Хранители: сегменты 0-{book1_end}")
    print(f"Две твердыни: сегменты {book1_end}-{book2_end}")
    print(f"Возвращение: сегменты {book2_end}-{len(segments)}")
    
    return {
        'text': text,
        'segments': segments,
        'segments_book1': segments_book1,
        'segments_book2': segments_book2,
        'segments_book3': segments_book3,
        'book1_end': book1_end,
        'book2_end': book2_end
    }

# Выполнение загрузки и разделения
BASE_DIR = Path.cwd()
data_path = BASE_DIR / 'Vlastelin_kolets.txt'
data = load_and_split_text(data_path)

# Извлекаем переменные для обратной совместимости
text = data['text']
segments = data['segments']
segments_book1 = data['segments_book1']
segments_book2 = data['segments_book2']
segments_book3 = data['segments_book3']
book1_end = data['book1_end']
book2_end = data['book2_end']

Найдены границы книг на позициях: [0, 819684, 1549962]

Хранители: 22 сегментов
Две твердыни: 21 сегментов
Возвращение государя: 19 сегментов
Всего смысловых сегментов (глав): 62

Границы:
Хранители: сегменты 0-22
Две твердыни: сегменты 22-43
Возвращение: сегменты 43-62


In [3]:
def initialize_natasha():
    """Инициализируем компоненты Natasha для извлечения сущностей"""
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    emb = NewsEmbedding()
    ner_tagger = NewsNERTagger(emb)
    morph_tagger = NewsMorphTagger(emb)
    return segmenter, morph_vocab, emb, ner_tagger, morph_tagger

def extract_entities_to_list(text, segmenter, morph_vocab, ner_tagger, morph_tagger):
    """Находим в тексте Персонажей (PER) и Локации (LOC)"""
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)
    
    for span in doc.spans:
        span.normalize(morph_vocab) # Приводим сущность к именительному падежу
    
    entities_data = []
    for span in doc.spans:
        if span.type in ['PER', 'LOC']:
            entities_data.append({
                'type': span.type,
                'raw_text': span.text.lower(),
                'normal_form': span.normal.lower()
            })
    
    return pd.DataFrame(entities_data)

def prepare_entities_for_review(df_entities):
    """Группируем найденные формы имен, чтобы было удобно проверить список"""
    review_df = df_entities.groupby(['normal_form', 'type']).agg({
        'raw_text': lambda x: ", ".join(set(x)), # Объединяем "арагорна, арагорну" в одну строку
        'type': 'count' # Считаем общее количество упоминаний
    }).rename(columns={'type': 'count'}).reset_index()
    
    # Переименовываем колонку с вариантами для наглядности
    review_df = review_df.rename(columns={'raw_text': 'variants_in_text'})
    
    # Сортируем по частоте
    review_df = review_df.sort_values(by='count', ascending=False)
    
    return review_df

def extract_and_save_entities(text, output_file='entities_to_review.csv'):
    """Полный цикл извлечения сущностей и сохранения в CSV"""
    segmenter, morph_vocab, emb, ner_tagger, morph_tagger = initialize_natasha()
    df_entities = extract_entities_to_list(text, segmenter, morph_vocab, ner_tagger, morph_tagger)
    df_to_excel = prepare_entities_for_review(df_entities)
    df_to_excel.to_csv(output_file, index=False, encoding='utf-8-sig')
    return df_to_excel

segmenter, morph_vocab, emb, ner_tagger, morph_tagger = initialize_natasha()

# Запускаем сбор
df_entities = extract_entities_to_list(text, segmenter, morph_vocab, ner_tagger, morph_tagger)
df_to_excel = prepare_entities_for_review(df_entities)

df_to_excel.to_csv('entities_to_review.csv', index=False, encoding='utf-8-sig')

C:\Users\Huawei\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pymorphy2\analyzer.py:114: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
def load_ner_mapping(csv_file='entities_to_review_new.csv'):
    """Загружаем словарь нормализации именованных сущностей из CSV"""
    df_cleaned = pd.read_csv(csv_file)
    
    # Создаем словарь замен
    # Нам нужно, чтобы каждый вариант вел к normal_form
    final_ner_mapping = {}
    
    for _, row in df_cleaned.iterrows():
        norm = row['normal_form'].replace(' ', '_')
        # Разбиваем строку с вариантами обратно в список
        variants = [v.strip() for v in str(row['variants_in_text']).split(',')]
        
        for v in variants:
            final_ner_mapping[v.lower()] = norm
    
    print(f"{len(final_ner_mapping)} форм имен и локаций.")
    return final_ner_mapping

my_mapping = load_ner_mapping()

708 форм имен и локаций.


In [5]:
my_mapping

{'фродо торбинс': 'фродо',
 'фродо': 'фродо',
 'фродо торбинса': 'фродо',
 'фродо торбинсу': 'фродо',
 'сэму': 'сэм',
 'сэма': 'сэм',
 'сэмом': 'сэм',
 'сэм': 'сэм',
 'сэме': 'сэм',
 'сэм скромби': 'сэм',
 'сэмом скромби': 'сэм',
 'сэма скромби': 'сэм',
 'сэму скромби': 'сэм',
 'сэммиум': 'сэм',
 'сэммиум скромби': 'сэм',
 'сын хэмбриджа': 'сэм',
 'сэммиума': 'сэм',
 'сэммиуму': 'сэм',
 'скромби': 'сэм',
 'гэндальфом': 'гэндальф',
 'гэндальфе': 'гэндальф',
 'гэндальфа': 'гэндальф',
 'гэндальф': 'гэндальф',
 'гэндальфу': 'гэндальф',
 'митрандир': 'гэндальф',
 'мирандиром': 'гэндальф',
 'митрандиру': 'гэндальф',
 'митрандира': 'гэндальф',
 'олорином': 'гэндальф',
 'старый маг': 'гэндальф',
 'гэндальф серый': 'гэндальф',
 'гэндальфа серого': 'гэндальф',
 'арагорн': 'арагорн',
 'арагорне': 'арагорн',
 'арагорну': 'арагорн',
 'арагорном': 'арагорн',
 'арагорна': 'арагорн',
 'колоброда': 'арагорн',
 'бродяжник': 'арагорн',
 'бродяжнику': 'арагорн',
 'бродяжником': 'арагорн',
 'бродяжника': '

In [6]:
def initialize_preprocessing(final_ner_mapping):
    """Очистка: только строчные буквы, удаление цифр и пунктуации"""
    morph = MorphAnalyzer()
    stopwords_ru = stopwords.words('russian') 
    common_stopwords = [
        'дело', 'место', "путь", "дорога", "направление", "сторона", "день", "ночь", "утро", "вечер", 
        "время", "час", "минута", "миля", "лига", "шаг", "рука", "нога", "глаз", 
        "голова", "плечо", "спина", "лицо", "дверь", "окно", "стол", "стена", 
        "голос", "звук", "слово", "дело", "мысль", "взгляд", "господин", "мастер", 
        "сударь", 'государь', 'князь', 'лестница', 'дом', 'гость', 'старик', 'народ',
        "мир", "история", "рассказ", "тайна", "книга", "средиземье", "война", 
        "помощь", "победа", "воинство", "потомок", "ответ", "память", "радость",
        "лестница", "дом", "гость", "старик", "народ", "слух", "мешок", "сто", "дурак",
        "вид", "конец", "начало", "число", "ряд", "надежда", "страх", "ужас", "смерть"
    ]
    all_stopwords = set(stopwords_ru + common_stopwords)
    return morph, all_stopwords

def preprocess_text(text, final_ner_mapping, morph, all_stopwords):
    # Базовая очистка
    text = text.lower()
    text = text.replace('-', ' ') # Убираем дефисы
    text = re.sub(r'\s+', ' ', text)

    sorted_keys = sorted(final_ner_mapping.keys(), key=len, reverse=True)
    for variant in sorted_keys:
        v_clean = variant.strip().lower().replace('-', ' ')
        if not v_clean: continue
        
        pattern = rf'\b{re.escape(v_clean)}\b'
        text = re.sub(pattern, final_ner_mapping[variant], text)
    
    # Удаляем всё, кроме букв и наших подчеркиваний в именах
    text = re.sub(r'[^а-яё\_\s]', ' ', text)
    
    words = text.split()
    tokens = []

    for word in words:
        # Пропускаем слишком короткие слова, если это не важные сущности
        if len(word) < 3 and word not in final_ner_mapping:
            continue
        # Определяем лемму
        if '_' in word:
            lemma = word
        elif word in final_ner_mapping:
            lemma = final_ner_mapping[word]
        else:
            parsed = morph.parse(word)[0]
            if 'NOUN' in parsed.tag: # Оставляем ТОЛЬКО существительные
                lemma = parsed.normal_form
            else:
                continue # Если не существительное - идем к следующему слову

        # Финальная проверка на стоп-слова
        if lemma not in all_stopwords:
            tokens.append(lemma)

    return tokens

def preprocess_segments_with_chunks(original_segments, final_ner_mapping, chunk_size=200, overlap=50):
    """Разбиваем главы на чанки с перекрытием для плавности анализа"""
    morph, all_stopwords = initialize_preprocessing(final_ner_mapping)
    all_processed_chunks = []
    chapter_start_indices = [] # Номера чанков, с которых начинаются главы
    current_global_idx = 0 # Счетчик общего количества чанков
    chunks_per_book = [0, 0, 0] 
        
    for idx, seg in enumerate(original_segments):
        # Запоминаем, какой по счету чанк является началом этой главы
        chapter_start_indices.append(current_global_idx)

        # Превращаем главу в список лемм
        tokens = preprocess_text(seg, final_ner_mapping, morph, all_stopwords)
        
        # Нарезка токенов главы на чанки
        chapter_chunks = []
        if len(tokens) <= chunk_size:
            if len(tokens) > 50:
                 chapter_chunks.append(tokens)
        else:
            step = chunk_size - overlap # Шаг смещения окна
            for i in range(0, len(tokens) - overlap, step):
                chunk = tokens[i : i + chunk_size]
                if len(chunk) > 50: 
                    chapter_chunks.append(chunk)
        
        # Добавляем чанки в общий список
        all_processed_chunks.extend(chapter_chunks)
        
        # Увеличиваем глобальный счетчик на количество чанков в главе
        current_global_idx += len(chapter_chunks)
        
        # Считаем, сколько чанков ушло на каждую из трех книг
        if idx < book1_end:
            chunks_per_book[0] += len(chapter_chunks)
        elif idx < book2_end:
            chunks_per_book[1] += len(chapter_chunks)
        else:
            chunks_per_book[2] += len(chapter_chunks)

    # Пересчитываем границы книг уже в масштабе чанков
    new_book1_end = chunks_per_book[0]
    new_book2_end = chunks_per_book[0] + chunks_per_book[1]

    print(f"Создано {len(all_processed_chunks)} чанков с перекрытием {overlap} токенов.")
    
    return all_processed_chunks, new_book1_end, new_book2_end, morph, all_stopwords, chapter_start_indices

processed, b1_end, b2_end, morph_obj, stops, chap_indices = preprocess_segments_with_chunks(segments, my_mapping)

data_to_save = {
    'segments': segments,
    'processed_segments': processed,
    'b1_end': b1_end,
    'b2_end': b2_end,
    'my_mapping': my_mapping,
    'chapter_to_chunk_indices': chap_indices,
    'segments_book1': segments_book1,
    'segments_book2': segments_book2,
    'segments_book3': segments_book3,
    'book1_end': book1_end,
    'book2_end': book2_end
}

with open('processed_data.pkl', 'wb') as f:
    pickle.dump(data_to_save, f)

Создано 495 чанков с перекрытием 50 токенов.
